# 💥 Chapter 16: PointNet - Advanced Deep Learning Classification

PointNet (Qi et al., 2017) revolutionized 3D Deep Learning by acting directly on points. It learns global features by aggregating local information using a symmetric function (Max Pooling).

**Workflow:**
1.  **Preparation**: Convert LAS files to HDF5 format (fast loading).
2.  **Architecture**: Implement PointNet in PyTorch.
3.  **Training**: Train the model to classify objects.
4.  **Inference**: Test on new data.

In [ ]:
import numpy as np
import laspy
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## 1. Data Preparation (HDF5)

Deep Learning requires fast I/O. We convert our raw LAS/PLY files into structured HDF5 files.

In [ ]:
def save_h5(h5_filename, data, label, data_dtype='float32', label_dtype='uint8'):
    h5_fout = h5py.File(h5_filename, 'w')
    h5_fout.create_dataset(
            'data', data=data,
            compression='gzip', compression_opts=4,
            dtype=data_dtype)
    h5_fout.create_dataset(
            'label', data=label,
            compression='gzip', compression_opts=1,
            dtype=label_dtype)
    h5_fout.close()
    print(f"Saved {h5_filename}")

# Example usage (commented out unless data exists)
# points = ... # [N_samples, 2048, 3]
# labels = ... # [N_samples]
# save_h5("../DATA/ply_data_train0.h5", points, labels)

## 2. PointNet Architecture (PyTorch)

A simplified implementation of the classification network.

In [ ]:
class PointNetCls(nn.Module):
    def __init__(self, k=2):
        super(PointNetCls, self).__init__()
        # Shared MLP (implemented as Conv1d with kernel size 1)
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        
        # Classification MLP
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
        
    def forward(self, x):
        # Input x: [Batch, 3, N_points]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Max Pooling (Symmetric Function)
        # Max over N_points dimension
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)

model = PointNetCls(k=5)
print(model)

## 3. Training Loop

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Dummy Training Step
inputs = torch.randn(32, 3, 2048) # Batch 32, 2048 points
labels = torch.randint(0, 5, (32,))

optimizer.zero_grad()
outputs = model(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

print(f"Dummy Loss: {loss.item():.4f}")